In [ ]:
%sql
SELECT * FROM `0a2f66c3e41f_df_geo`;


ind,country,coordinates,timestamp
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
10794,Cocos (Keeling) Islands,"List(-89.5236, -154.567)",2022-01-01T02:26:50.000+0000
2074,Central African Republic,"List(-52.3213, -50.11)",2019-11-03T05:41:59.000+0000
9979,Dominican Republic,"List(14.9967, -120.682)",2018-07-18T19:01:46.000+0000
2923,Cote d'Ivoire,"List(-84.6302, -164.507)",2019-09-08T22:53:09.000+0000
8304,French Guiana,"List(-28.8852, -164.87)",2019-09-13T04:50:29.000+0000
4315,Cote d'Ivoire,"List(-45.8508, 66.1003)",2019-12-15T03:51:28.000+0000
5069,Azerbaijan,"List(-63.0063, -157.474)",2021-03-20T09:32:44.000+0000
6145,Mozambique,"List(-65.9079, -143.845)",2019-12-05T02:09:44.000+0000
428,Bangladesh,"List(-83.4105, -150.788)",2020-09-27T18:46:41.000+0000


In [ ]:
from pyspark.sql.functions import udf, col, array, to_timestamp
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.sql import SparkSession

# File location and type
file_location = "/mnt/0a2f66c3e41f.df_geo/topics/0a2f66c3e41f.geo/partition=0/*.json"
file_type = "json"

# # Ask Spark to infer the schema
infer_schema = "true"

# Read in JSONs from mounted S3 bucket
df = spark.read.format(file_type).option("inferSchema", infer_schema).load(file_location)

# Create a new column "Coordinates" based on Latitude and Longitude
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
df_with_coordinates = df.withColumn("coordinates", array(col("latitude"), col("longitude")))

# Drop the latitude and longitude columns from the DataFrame
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
columns_to_drop = ["latitude", "longitude"]
df_without_coordinates = df_with_coordinates.drop(*columns_to_drop)

# Change timestamp datatype to timestamp
df_with_timestamp = df_without_coordinates.withColumn("timestamp", col("timestamp").cast("timestamp"))

# Reorder the DataFrame columns
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
df_with_timestamp = df_with_timestamp.withColumnRenamed('index', 'ind')
new_order = ["ind", "country", "coordinates", "timestamp"]
df_reordered = df_with_timestamp.select(*new_order)

df_reordered.printSchema()
display(df_reordered)

# Write dataframe to a temporary view
df_reordered.createOrReplaceTempView("0a2f66c3e41f_df_geo")

# Create table from temporary view
spark.sql("CREATE TABLE 0a2f66c3e41f_df_geo AS SELECT * FROM 0a2f66c3e41f_df_geo")


root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = false)
 |-- element: double (containsNull = true)
-- timestamp: timestamp (nullable = true)

ind,country,coordinates,timestamp
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
10794,Cocos (Keeling) Islands,"List(-89.5236, -154.567)",2022-01-01T02:26:50.000+0000
2074,Central African Republic,"List(-52.3213, -50.11)",2019-11-03T05:41:59.000+0000
9979,Dominican Republic,"List(14.9967, -120.682)",2018-07-18T19:01:46.000+0000
2923,Cote d'Ivoire,"List(-84.6302, -164.507)",2019-09-08T22:53:09.000+0000
8304,French Guiana,"List(-28.8852, -164.87)",2019-09-13T04:50:29.000+0000
4315,Cote d'Ivoire,"List(-45.8508, 66.1003)",2019-12-15T03:51:28.000+0000
5069,Azerbaijan,"List(-63.0063, -157.474)",2021-03-20T09:32:44.000+0000
6145,Mozambique,"List(-65.9079, -143.845)",2019-12-05T02:09:44.000+0000
428,Bangladesh,"List(-83.4105, -150.788)",2020-09-27T18:46:41.000+0000


Out[3]: DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]